In [2]:
#import required libraries
import requests
from bs4 import BeautifulSoup

# MSNBC Web-Scraping

## Step 1: Get the data from the home page of the transcripts website
- get a list of the dates across the entire dataset
- create a list and just iterate through the list and append the dataset to get 

Source on web-scraping: https://realpython.com/beautiful-soup-web-scraper-python/#step-3-parse-html-code-with-beautiful-soup 


## Get to the website and convert to a soup object

In [8]:
#step 1: store the URL
rachel_maddow_URL = "https://www.msnbc.com/transcripts/show/rachel-maddow-show"

#step 2: get to the landing page of the website
landing_page = requests.get(rachel_maddow_URL)

#step 3: convert page into html
soup = BeautifulSoup(landing_page.content, "html.parser")

#below is a potential alternative additional link to look at/start with
#https://www.msnbc.com/transcripts/show/rachel-maddow-show?dateRange=2020-03-01+TO+2022-03-31 <- all show transcripts from 3/1/2020 to 3/31/2022


## Get the Transript Feed (A collection of individual transcript cards)

In [16]:
#<div class="transcripts-page">
#<div class="transcripts-feed">


#step 4: extract the transcript feed page part of the soup
transcript_feed = soup.find(class_="transcripts-feed")

#print out example of the html output that is contained in transcript feed
print(transcript_feed.prettify()[:500])


<div class="transcripts-feed">
 <div class="transcript-card">
  <div class="transcript-card__air-date">
   Mar 16 2022
  </div>
  <div class="transcript-card__details">
   <a class="transcript-card__show-name" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">
    The Rachel Maddow Show
   </a>
   <a class="transcript-card__headline" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">


## Extract Individual Transcript Cards 

In [17]:
#step 5: get all of the transcript cards -- creates an object that is iterable to look at
transcript_cards = transcript_feed.find_all("div", class_ ="transcript-card")

In [20]:
#step 6: display the results of the above
for card in transcript_cards[:1]:
    print(card, end="\n"*2)

<div class="transcript-card"><div class="transcript-card__air-date">Mar 16 2022</div><div class="transcript-card__details"><a class="transcript-card__show-name" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">The Rachel Maddow Show</a><a class="transcript-card__headline" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">Transcript: The Rachel Maddow Show, 3/15/22</a><span class="transcript-card__guests"><span class="">Guests: Serhiy Leshchenko, Amy Klobuchar, Alexander Goncharov</span></span></div></div>



In [12]:
'''
Key Components of HTML output:
Each entry = a "transcript card"
Note: the transcript feed is made up of many transcript cards

Transcript Card (<div class="transcript-card">)
    Air Date: <div class="transcript-card__air-date">
    Transcript Card Details:  <div class="transcript-card__details">
        Show Name: <a class="transcript-card__show-name" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">The Rachel Maddow Show</a>
        Headline: <a class="transcript-card__headline" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">Transcript: The Rachel Maddow Show, 3/15/22</a> 
        Guests: <span class="transcript-card__guests"><span class="">Guests: Serhiy Leshchenko, Amy Klobuchar, Alexander Goncharov</span></span></div></div>


'''


'\nKey Components of HTML output:\nEach entry = a "transcript card"\nNote: the transcript feed is made up of many transcript cards\n\nTranscript Card (<div class="transcript-card">)\n    Air Date: <div class="transcript-card__air-date">\n    Transcript Card Details:  <div class="transcript-card__details">\n        Show Name: <a class="transcript-card__show-name" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">The Rachel Maddow Show</a>\n        Headline: <a class="transcript-card__headline" href="https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030">Transcript: The Rachel Maddow Show, 3/15/22</a> \n        Guests: <span class="transcript-card__guests"><span class="">Guests: Serhiy Leshchenko, Amy Klobuchar, Alexander Goncharov</span></span></div></div>\n\n\n\n\n'

In [ ]:
'''
Example URLS:
https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-15-22-n1292030 
https://www.msnbc.com/transcripts/rachel-maddow-show/transcript-rachel-maddow-show-3-14-22-n1292021


'''